# Assignment 1: Problem B
In this problem, you will implement a simple Naive Bayes classifier to classifiy mails as spam or not. You will need to create a 7-fold cross validation to train and test your model. You may choose to discard various stop words, commas, fullstops, numbers, hyphens, brackets, exclamation marks and any other single/double letter words (such as a, an, the, be etc) which do not contribute to the sentiment of the text.Use laplace smoothening to avoid the problem of divison by zero.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import string
import re 

In [2]:
def process_data(fname):
    df = pd.read_csv(fname,sep='\n',header=None)
    data=np.array(df)
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    proc_data=[]
   
    for row in range(len(data)):
        for ele in data[row][0]:  
            if (ele in punc):  
                data[row][0] = data[row][0].replace(ele," ")
    for row in range(len(data)):
        proc_data.append(data[row][0].split())
    for row in range(len(proc_data)):
       
        for i in range(len(proc_data[row])):
            proc_data[row][i]=proc_data[row][i].lower()
    
    for i in range(len(proc_data)):
        j=0
        while (j<len(proc_data[i])):
            if(len(proc_data[i][j])<=2 and (proc_data[i][j]!='1' and proc_data[i][j]!='0')):
                proc_data[i].remove(proc_data[i][j])
                j=j-1 
            j+=1
    

    return proc_data

In [3]:
def crossvalidation(data,f):
    testset={}
    trainset={}
    size=len(data)
    for i in range(f):
        testset[i]=data[(int)(i*size/f):(int)((i+1)*size/f)]
        trainset[i]=data[0:(int)((i)*size/f)]+data[(int)((i+1)*size/f):size]
    return testset,trainset

In [4]:
def make_dataset(values):
    dataset={}
    count=np.zeros(2)
    c1=np.zeros(2)
    for i in range(len(values)):
        c1[int(values[i][-1])]+=1
        for word in values[i][0:-1]:
            pair=(word,int(values[i][-1]))
            count[int(values[i][-1])]+=1
            if pair in dataset:
                dataset[pair]+=1
            else:
                dataset[pair]=1
            
    return dataset,count,c1

In [5]:
def prob(fo,to,d,a):
    pro=float((fo+a)/(to+(a*d)))
    return pro

In [6]:
def naive_bayes(train,alpha):
    dataset,count,c1=make_dataset(train)
    n=np.array([keys[0] for keys in dataset.keys()])
    n=np.unique(n)
    d=len(n)
    for key in dataset.keys():
        if key[1]==1:
            dataset[key]=prob(dataset[key],count[1],d,alpha)
        else:
            dataset[key]=prob(dataset[key],count[0],d,alpha)
    
    return dataset,count,c1,d       
            
    

In [7]:
def testing_nb(train,test):
    dataset,count,c1,d=naive_bayes(train,1)
    p0=c1[0]/((c1[0]+c1[1]))
    predict=[]
    for i in range(len(test)):
        prob0=p0
        prob1=1-p0
        for word in test[i][0:-1]:
            pair0=(word,0)
            pair1=(word,1)
            if pair0 in dataset:
                prob0=prob0*dataset[pair0]
            else:
                prob0=prob0*prob(0,count[0],d,1)
            if pair1 in dataset:
                prob1=prob1*dataset[pair1]
            else:
                prob1=prob1*prob(0,count[1],d,1)
            
        if(prob0>prob1):
            predict.append(0)
        else:
             predict.append(1)
    correct=0
    for i in range(len(test)):
        if int(test[i][-1])==predict[i]:
            correct+=1
    accuracy=float((correct*100)/len(test))
    return accuracy
  

In [8]:
fname='/Users/durbasatpathi/Desktop/ml/dataset_NB.txt'
data=process_data(fname)
test,train = crossvalidation(data,7)
accuracy=0
for i in range(7):
    a=testing_nb(train[i],test[i])
    accuracy+=a
    print('Accuracy over fold ',i+1,':   ',a,'%')
print('Overall average accuracy: ',accuracy/7,'%')



Accuracy over fold  1 :    80.98591549295774 %
Accuracy over fold  2 :    86.01398601398601 %
Accuracy over fold  3 :    82.51748251748252 %
Accuracy over fold  4 :    81.81818181818181 %
Accuracy over fold  5 :    80.41958041958041 %
Accuracy over fold  6 :    82.51748251748252 %
Accuracy over fold  7 :    78.32167832167832 %
Overall average accuracy:  81.7991867287642 %
